---
title: IDs from STEREO
format:
  html:
    code-fold: true
output-file: stereo.html
---

## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create stearo
```

To get STEREO-A state data, run `kedro run --to-outputs=sta.primary_state_rtn_1h`

To get candidates data, run `kedro run --from-inputs=sta.feature_1s --to-outputs=candidates.sta_1s`

In [ ]:
#| default_exp pipelines/stearo/pipeline

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| code-summary: import all the packages needed for the project
#| export
#| output: hide

from fastcore.utils import *
from fastcore.test import *
from ids_finder.utils.basic import *
from ids_finder.core import *

import polars as pl
import pandas
import numpy as np

from datetime import timedelta
from loguru import logger

from typing import Callable, Dict

[10/24/23 14:51:05] WARNING  TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook    ]8;id=317044;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=51268;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter                  
                             console)                                                                              
                                                                                                                   

In [ ]:
#| export
from ids_finder.core import extract_features

#### `Kerdo`

In [ ]:
#| export
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

In [ ]:
#| eval: false
from ids_finder.utils.basic import load_catalog

catalog = load_catalog()

[10/24/23 14:51:06] WARNING  KedroDeprecationWarning: 'PartitionedDataset' has been moved to        ]8;id=273524;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=241962;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             `kedro-datasets` and will be removed in Kedro 0.19.0.                                 
                                                                                                                   

                    WARNING  KedroDeprecationWarning: 'PartitionedDataset' has been moved to        ]8;id=762659;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=335193;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             `kedro-datasets` and will be removed in Kedro 0.19.0.                                 
                                                                                                                   

                    WARNING  KedroDeprecationWarning: 'AbstractVersionedDataSet' has been renamed   ]8;id=194900;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=634475;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             to 'AbstractVersionedDataset', and the alias will be removed in Kedro                 
                             0.19.0                                                                                
                                                                                                                   

## Magnetic field data pipeline

STEREO magnetic field is already in RTN coordinates, so no need to transform it.

Download data using `pyspedas`, but load it using `pycdfpp` (using `pyspedas` to load the data directly into `xarray` is very slow)

### Downloading data

In [ ]:
#| export
import os
os.environ['SPEDAS_DATA_DIR'] = f"{os.environ['HOME']}/data"
import pyspedas

In [ ]:
# | export
def download_mag_data(
    start: str = None,
    end: str = None,
    probe: str = "a",
) -> Iterable[str]:
    trange = [start, end]
    files = pyspedas.stereo.mag(trange, downloadonly=True, probe=probe)
    return files

### Preprocessing data

In [ ]:
#| export
from ids_finder.utils.basic import cdf2pl
from pipe import select
from typing import Iterable

In [ ]:
# | export
def preprocess_mag_data(
    raw_data: Iterable[str] = None, # List of CDF files
    ts: str = "1s",  # time resolution
) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Downsample the data to a given time resolution
    - Applying naming conventions for columns
    """
    every = pandas.Timedelta(ts)
    period = 2 * every

    df: pl.LazyFrame = pl.concat(raw_data | pmap(cdf2pl, var_name="BFIELD"))

    return (
        df.pipe(resample, every=every, period=period)
        .rename(
            {
                "BFIELD_0": "b_r",
                "BFIELD_1": "b_t",
                "BFIELD_2": "b_n",
                "BFIELD_3": "b_mag",
            }
        )
        .collect()
    )

### Processing data

In [ ]:
#| export
def process_mag_data(
    raw_data: pl.DataFrame,
    ts: str = None,  # time resolution
    coord: str = None,
) -> Dict[str, pl.DataFrame]:
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Partitioning data, for the sake of memory
    """
    return partition_data_by_year(raw_data)

### Pipeline

In [ ]:
#| export
def create_mag_data_pipeline(
    sat_id,
    ts: str = "1s",  # time resolution,
    tau: str = "60s",  # time window
    **kwargs,
) -> Pipeline:
    node_download_data = node(
        download_mag_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
        ),
        outputs=f"raw_mag_files",
        name=f"download_{sat_id.upper()}_magnetic_field_data",
    )

    node_preprocess_data = node(
        preprocess_mag_data,
        inputs=dict(
            raw_data=f"raw_mag_files",
        ),
        outputs=f"inter_mag_{ts}",
        name=f"preprocess_{sat_id.upper()}_magnetic_field_data",
    )

    node_process_data = node(
        process_mag_data,
        inputs=f"inter_mag_{ts}",
        outputs=f"primary_mag_{ts}",
        name=f"process_{sat_id.upper()}_magnetic_field_data",
    )

    node_extract_features = node(
        extract_features,
        inputs=[f"primary_mag_{ts}", "params:tau", "params:extract_params"],
        outputs=f"feature_tau_{tau}",
        name=f"extract_{sat_id}_features",
    )

    nodes = [
        node_download_data,
        node_preprocess_data,
        node_process_data,
        node_extract_features,
    ]

    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
            "params:tau": "params:tau",
        },
    )

    return pipelines

## State data pipeline

For STEREO's mission, We use 1-hour averaged merged data from [COHOWeb](https://omniweb.gsfc.nasa.gov/coho/).

See [STEREO ASCII merged data](https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/merged/aareadme_sta) and one sample file [here](https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/merged/stereoa2011.asc

Plasma in RTN (Radial-Tangential-Normal) coordinate system
- Proton Flow Speed, km/sec
- Proton Flow Elevation Angle/Latitude, deg.
- Proton Flow Azimuth Angle/Longitude, deg.
- Proton Density, n/cc
- Proton Temperature, K)

Notes
- Note1: There is a big gap 2014/12/16 - 2015/07/20 in plasma data
- Note2: There is a big gap 2015/03/21 - 2015/07/09 and 2015/10/27 - 2015/11/15 in mag data
- Note that for missing data, fill values consisting of a blank followed by 9's which together constitute the format are used

### Loading data

In [ ]:
#| export
import pooch
from pipe import select

In [ ]:
# | export
def download_state_data(
    start: str = None,
    end: str = None,
) -> List[str]:
    download = partial(pooch.retrieve, known_hash=None)

    start_time = pandas.Timestamp(start)
    end_time = pandas.Timestamp(end)

    url = "https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/merged/stereoa{year}.asc"

    files = list(
        range(start_time.year, end_time.year + 1)
        | select(lambda x: url.format(year=x))
        | select(download)
    )
    return files

In [ ]:
#| export
headers = """Year
DOY
Hour
Radial Distance, AU
HGI Lat. of the S/C
HGI Long. of the S/C
IMF BR, nT (RTN)
IMF BT, nT (RTN)
IMF BN, nT (RTN)
IMF B Scalar, nT
SW Plasma Speed, km/s
SW Lat. Angle RTN, deg.
SW Long. Angle RTN, deg.
SW Plasma Density, N/cm^3
SW Plasma Temperature, K
1.8-3.6 MeV H flux,LET
4.0-6.0 MeV H flux,LET
6.0-10.0 MeV H flux, LET
10.0-12.0 MeV H flux,LET
13.6-15.1 MeV H flux, HET
14.9-17.1 MeV H flux, HET
17.0-19.3 MeV H flux, HET
20.8-23.8 MeV H flux, HET
23.8-26.4 MeV H flux, HET
26.3-29.7 MeV H flux, HET
29.5-33.4 MeV H flux, HET
33.4-35.8 MeV H flux, HET
35.5-40.5 MeV H flux, HET
40.0-60.0 MeV H flux, HET
60.0-100.0 MeV H flux, HET
0.320-0.452 MeV H flux, SIT
0.452-0.64 MeV H flux, SIT
0.640-0.905 MeV H flux, SIT
0.905-1.28 MeV H flux, SIT
1.280-1.81 MeV H flux, SIT
1.810-2.56 MeV H flux, SIT
2.560-3.62 MeV H flux, SIT"""

def load_state_data(
    start: str = None,
    end: str = None,
) -> pl.DataFrame:
    """
    - Downloading data
    - Reading data into a proper data structure, like dataframe.
        - Parsing original data (dealing with delimiters, missing values, etc.)
    """
    files = download_state_data(start, end)
    
    labels = headers.split("\n")
    missing_values = ["999.99", "9999.9", "9999999."]

    df = pl.concat(
        files
        | pmap(
            pandas.read_csv,
            delim_whitespace=True,
            names=labels,
            na_values=missing_values,
        )
        | select(pl.from_pandas)
    )
    
    return df


### Preprocessing data

In [ ]:
#| export
def preprocess_state_data(
    raw_data: pl.DataFrame,
    ts: str = None,  # time resolution
    coord: str = None,
) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Parsing and typing data (like from string to datetime for time columns)
    - Changing storing format (like from `csv` to `parquet`)
    """

    return raw_data.with_columns(
        time=(
            pl.datetime(pl.col("Year"), month=1, day=1)
            + pl.duration(days=pl.col("DOY") - 1, hours=pl.col("Hour"))
        ).dt.cast_time_unit("ns"),
    )

#### Processs state data

In [ ]:
# | export
def convert_state_to_rtn(df: pl.DataFrame) -> pl.DataFrame:
    """Convert state data to RTN coordinates"""
    plasma_speed = pl.col("plasma_speed")
    sw_elevation = pl.col("sw_elevation").radians()
    sw_azimuth = pl.col("sw_azimuth").radians()
    return df.with_columns(
        sw_vel_r=plasma_speed * sw_elevation.cos() * sw_azimuth.cos(),
        sw_vel_t=plasma_speed * sw_elevation.cos() * sw_azimuth.sin(),
        sw_vel_n=plasma_speed * sw_elevation.sin(),
    ).drop(["sw_elevation", "sw_azimuth"])


STATE_POSITION_COLS = [
    "Radial Distance, AU",
    "HGI Lat. of the S/C",
    "HGI Long. of the S/C",
]

STATE_PLASMA_COLS = [
    "SW Plasma Speed, km/s",
    "SW Lat. Angle RTN, deg.",
    "SW Long. Angle RTN, deg.",
    "SW Plasma Density, N/cm^3",
    "SW Plasma Temperature, K",
]

columns_name_mapping = {
    "SW Plasma Speed, km/s": "plasma_speed",
    "SW Lat. Angle RTN, deg.": "sw_elevation",
    "SW Long. Angle RTN, deg.": "sw_azimuth",
    "SW Plasma Density, N/cm^3": "plasma_density",
    "SW Plasma Temperature, K": "plasma_temperature",
    "Radial Distance, AU": "radial_distance",
}


def process_state_data(
    df: pl.DataFrame, columns: list[str] = STATE_POSITION_COLS + STATE_PLASMA_COLS
) -> pl.DataFrame:
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Applying naming conventions for columns
    - Transforming data to RTN (Radial-Tangential-Normal) coordinate system
    - Discarding unnecessary columns
    """

    return (
        df.select("time", *columns)
        .rename(columns_name_mapping)
        .pipe(convert_state_to_rtn)
        .rename(
            {
                "sw_vel_r": "v_x",
                "sw_vel_t": "v_y",
                "sw_vel_n": "v_z",
            }
        )
    )

### Pipelines

In [ ]:
# | export
def create_state_data_pipeline(
    sat_id = 'sta',
    ts: str = '1h',  # time resolution
    **kwargs
) -> Pipeline:
    
    node_load_data = node(
        load_state_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
        ),
        outputs=f"raw_state",
        name=f"load_{sat_id.upper()}_state_data",
    )

    node_preprocess_data = node(
        preprocess_state_data,
        inputs=dict(
            raw_data=f"raw_state",
        ),
        outputs=f"inter_state_{ts}",
        name=f"preprocess_{sat_id.upper()}_state_data",
    )
    
    node_process_data = node(
        process_state_data,
        inputs=f"inter_state_{ts}",
        outputs=f"primary_state_{ts}",
        name=f"process_{sat_id.upper()}_state_data",
    )
    
    nodes = [
        node_load_data,
        node_preprocess_data,
        node_process_data,
    ]
    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
        },
    )

    return pipelines

## Pipelines

In [ ]:
#| export
from ids_finder.candidates import create_candidate_pipeline

In [ ]:
# | export
def create_pipeline(
    sat_id="sta",
    tau="60s",
    ts_mag="1s",  # time resolution of magnetic field data
    ts_state="1h",  # time resolution of state data
) -> Pipeline:
    return (
        create_mag_data_pipeline(sat_id, ts=ts_mag, tau=tau)
        + create_state_data_pipeline(sat_id, ts=ts_state)
        + create_candidate_pipeline(sat_id, tau=tau, ts_state=ts_state)
    )

## Obsolete codes

NOTE: one can also use `speasy` to download data, however this is slower for `STEREO` data.

In [ ]:
%%markdown
sat_fgm_product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD
sat_fgm_product = 'cda/STA_L1_MAG_RTN/BFIELD'
products = [sat_fgm_product]

dataset = spz.get_data(products, test_trange, disable_proxy=True)
sat_fgm_data  = dataset[0]
data_preview(sat_fgm_data)

Download data in a background thread

In [ ]:
%%markdown

@threaded
def download_data(products, trange):
    logger.info("Downloading data")
    spz.get_data(products, trange, disable_proxy=True)
    logger.info("Data downloaded")
    
download_data(products, trange)

In [ ]:
import speasy as spz

In [ ]:
cda_tree: spz.SpeasyIndex = spz.inventories.tree.cda
product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN

logger.info(product.description)
logger.info(product.ID)
logger.info(product.BFIELD.CATDESC)
logger.info(product.BFIELD.spz_uid())

# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.
# spz.inventories.data_tree.cda.STEREO.STEREOA.IMPACT_MAG.STA_LB_MAG_RTN.description
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.MAGFLAGUC.CATDESC
spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.CATDESC
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.